# Tensorflow Keras Boston House Data

#### Disable Tensorflow Warnings

In [1]:
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed 
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#### Import der Bibliotheken

In [2]:
import numpy as np
import pandas as pd
import os
import importlib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.datasets import boston_housing

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [3]:
# Import weiterer py-files, Aufruf mit utils.xyz
!wget -O utils.py https://raw.githubusercontent.com/dsbademo/DSBA/main/utils.py
utils = importlib.import_module("utils", "./utils.py")

--2022-09-28 18:02:58--  https://raw.githubusercontent.com/dsbademo/DSBA/main/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2384 (2.3K) [text/plain]
Saving to: 'utils.py'

utils.py            100%[===================>]   2.33K  --.-KB/s    in 0s      

2022-09-28 18:02:59 (5.44 MB/s) - 'utils.py' saved [2384/2384]



#### Laden der Daten

In [4]:
header_list = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']
df_houses = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data', names = header_list, sep='\s+')

#### Exploration der Daten

In [5]:
df_houses.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7


####  Train und Test Data

In [6]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [7]:
print(train_data.shape)
print(test_data.shape)
print(train_targets[0:10])

(404, 13)
(102, 13)
[15.2 42.3 50.  21.1 17.7 18.5 11.3 15.6 15.6 14.4]


#### Standardisierung der Daten

In [8]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

#### Baseline NN-Modell

In [9]:
def baseline_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mse','mae'])
    return model

#### Large NN-Modell

In [10]:
def larger_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mse','mae'])
    return model

#### Training

In [11]:
num_epochs = 50

In [12]:
network = baseline_model()
baseline_history = network.fit(train_data, train_targets, epochs=num_epochs, batch_size=1, verbose=0, validation_data=(test_data, test_targets))

Metal device set to: Apple M2


In [ ]:
network = larger_model()
large_history = network.fit(train_data, train_targets, epochs=num_epochs, batch_size=1, verbose=0, validation_data=(test_data, test_targets))

In [ ]:
print(baseline_history.history['loss'][0:5])
print(large_history.history['loss'][0:5])

#### Small Model

In [ ]:
base_loss = baseline_history.history['loss']
base_val_loss = baseline_history.history['val_loss']
utils.plot_history(base_loss, base_val_loss, 'training', 'test')

#### Large Model

In [ ]:
large_loss = large_history.history['loss']
large_val_loss = large_history.history['val_loss']
utils.plot_history(large_loss, large_val_loss, 'training', 'test')